In [2]:
#### Prod APP ###
### This app makes prediction of customer churn. You need to upload data for prediction, a visualisation will be shown and one can download file with predictions.
### This app requires assets folder (which holds pictures, styling etc), prod_model file which should be a pickled model
### Drag and drop button is where you need to drop data
### Run model button will load model, run predictions & make visulisations
### Download csv downloads a file with data and prediction of churn and the model probability of a client belonging to churn class

from dash import Dash, html, dcc, Output, Input, State, callback, dash_table
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

import pickle

import base64
import io
import datetime

app = Dash(__name__,external_stylesheets=[dbc.themes.DARKLY])

side_bar_style={
            'height': '60px',
            'lineHeight': '60px',
            'width':'300px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '10px',
            'textAlign': 'center',
            'margin': '0px',
            'background-color': 'rgba(255,0,0, 0)',
            'border-color': 'white',
            'color': 'white',
            'display':'flex',
            'justify-content': 'center'
            }

cards_style={'margin': '10px',"background-color": 'rgba(255,0,0, 0)'} #transparent color for card body and a bit of margin
card_header_style={"background-color": 'rgba(255,0,0, 0)','textAlign': 'center'} #transparent color for card heard
kpi_card_style={"background-color": 'rgba(255,0,0, 0)','textAlign': 'center', 'padding': '5px','margin':'15px'}
kpi_header_style={"background-color": 'rgba(255,0,0, 0)'}

fig_style={'plot_bgcolor': '#39618b','paper_bgcolor': '#375a7f', 'xaxis_title':'','yaxis_title':'','title_x':0.5}
graph_style={}
button_style={'width':'50'}

### define cards for top bar and respective callbacks###

###Upload card & callback ####
##################################################################################################################
upload_card= html.Div([html.Br , dcc.Upload(id='upload-data', children=html.Div(['Drag & Drop Input']), style=side_bar_style, multiple=True)])
upload_card= dbc.Card([
        dbc.CardHeader('Load data for prediction',style=card_header_style),
        dbc.CardBody(dcc.Upload(id='upload-data', children=html.Div(['Drag & Drop Input']),
        style=side_bar_style, # Allow multiple files to be uploaded
        multiple=True))
            ],style=cards_style
            )

### It reads the file loaded and saves it to the store where other callback can access it ###
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df1 = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df1 = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])
    return df1

@app.callback(
            Output('store-data', 'data'),
            # Output('table_tab','children'),
            Input('upload-data', 'contents'),
            State('upload-data', 'filename'),
            State('upload-data', 'last_modified'),
            prevent_initial_call=True)
            
def update_output(list_of_contents, list_of_names,date_):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n) for c, n in
            zip(list_of_contents, list_of_names)]
        return children[0].to_dict('records')
##################################################################################################################

###Run model card & callback ####
run_card=dbc.Card([
        dbc.CardHeader('Run Prediction',style=card_header_style),
        dbc.CardBody([
            html.Button("Run Model", id="btn_model",
            style=side_bar_style
            ),
            dcc.Download(id="download-dataframe")])
            ],style=cards_style
            )

@app.callback(
    Output("store-prediction", "data"),
    Input("btn_model", "n_clicks"),
    State('store-data','data'),
    prevent_initial_call=True,
)
def func(n_clicks,data):
    #load data from store
    dff=pd.DataFrame(data)
    #clean data from store
    dff=dff.drop(columns=["customerID"])
    dff.rename(columns={'Churn':'Orig_Churn'},inplace=True)
    dff['TotalCharges']=pd.to_numeric(dff['TotalCharges'],errors='coerce')
    dff.dropna(inplace=True)
    #load model & make prediction
    model = pickle.load(open('prod_model', 'rb'))
    dff['Churn']=model.predict(dff)
    dff['Churn'].replace((1, 0), ('yes', 'no'), inplace=True)
    dff['Churn probability']=model.predict_proba(dff)[:,1]
    return dff.to_dict('records')


### Download Card
#######################################################################################################################
download_card=dbc.Card([
        dbc.CardHeader('Download Prediction',style=card_header_style),
        dbc.CardBody([
            html.Button("Download CSV", id="btn_download",
            style=side_bar_style
            ),
            ])
            ],style=cards_style
            )

@app.callback(
    Output("download-dataframe", "data"),
    Input("btn_download", "n_clicks"),
    State('store-prediction','data'),
    prevent_initial_call=True,
)
def func(n_clicks,data):
    #load data from store
    dff=pd.DataFrame(data)
    return dcc.send_data_frame(dff.to_csv, "mydf.csv")
######################################################################################################################

welcome_hor_bar=dbc.Col([upload_card,run_card,download_card],width=2)

### Define top bar ###
welcome_top_bar=dbc.Row([dbc.Col(width=3),dbc.Col(upload_card,width=2),dbc.Col(run_card,width=2),dbc.Col(download_card,width=2)],style={'background-image': 'url("/assets/Background.svg")'})

### Define bottom part
welcome_content = dbc.Row([
    dbc.Row(id='graph1',class_name='row g-0',style={'background-color':'#375a7f'}),
    dbc.Row(id='table-placeholder', children=[],class_name='row g-0',style={'background-color':'#375a7f'}),
    # Will use store to keep the data uploaded by users & predictions
    dcc.Store(id='store-data', data=[], storage_type='memory'), # 'local' or 'session'
    dcc.Store(id='store-prediction', data=[], storage_type='memory') # 'local' or 'session'
],class_name='row g-0',style={'background-color':'#375a7f'})

# #Wrap blocks into one
mass_pred_vertical = html.Div([welcome_top_bar,welcome_content],style={'background-color':'#375a7f'},className="row no-gutters")
# mass_pred_horiz = html.Div([welcome_hor_bar,welcome_content],style={'background-color':'#375a7f'},className="row no-gutters")

single_prediction=html.Div()

tabs_=dbc.Tabs(id='tabs',children=[dbc.Tab(label='Mass prediction',children=[welcome_top_bar]),dbc.Tab(label='Single prediction',children=single_prediction)])

app.layout = html.Div(mass_pred_vertical)

### This creates a graph when store data changes ###
@callback(
    Output('graph1', 'children'),
    Input('store-prediction', 'data'),
    prevent_initial_call=True
)
def create_graph1(data):
    df=pd.DataFrame(data)
    # df['Churn'].replace((1, 0), ('yes', 'no'), inplace=True)
    kpi0= dbc.Col(dbc.Card([dbc.CardHeader('Total Clients Count',style=kpi_header_style),dbc.CardBody(html.H3(len(df)))],style=kpi_card_style))
    kpi1= dbc.Col(dbc.Card([dbc.CardHeader('Total Churn Count',style=kpi_header_style),dbc.CardBody(html.H3(len(df[df['Churn']=='yes'])))],style=kpi_card_style))
    kpi2= dbc.Col(dbc.Card([dbc.CardHeader('Total Churn %',style=kpi_header_style),dbc.CardBody(html.H3('{:.1%}'.format(len(df[df['Churn']=='yes'])/len(df))))],style=kpi_card_style))
    kpi3= dbc.Col(dbc.Card([dbc.CardHeader('Total Monthly Income',style=kpi_header_style),dbc.CardBody(html.H3(df['MonthlyCharges'].sum()))],style=kpi_card_style))
    kpi4= dbc.Col(dbc.Card([dbc.CardHeader('Monthly Income Churn',style=kpi_header_style),dbc.CardBody(html.H3(df[df['Churn']=='yes']['MonthlyCharges'].sum()))],style=kpi_card_style))
    kpi5= dbc.Col(dbc.Card([dbc.CardHeader('% Monthly Income Churn',style=kpi_header_style),dbc.CardBody(html.H3('{:.1%}'.format(df[df['Churn']=='yes']['MonthlyCharges'].sum()/df['MonthlyCharges'].sum())))],style=kpi_card_style))
    # kpi1=dbc.Col(dbc.CardBody('Total Churn {} {:.1%}'.format(len(df[df['Churn']=='yes']),len(df[df['Churn']=='yes'])/len(df))))
    # kpi2=dbc.Col(dbc.CardBody('Total Churn {} {:.1%}'.format(df[df['Churn']=='yes']['MonthlyCharges'].sum(),df[df['Churn']=='yes']['MonthlyCharges'].sum()/df['MonthlyCharges'].sum())))
    kpis=dbc.Row([kpi0,kpi1,kpi2,kpi3,kpi4,kpi5])
    #Plots showing proportion of predicted churn
    fig1=px.histogram(df,x='MonthlyCharges',color="Churn",title='Predicted churn count by monthly charges',color_discrete_sequence=['indianred','darkseagreen'],nbins=30,template='plotly_dark')
    fig1.update_layout(fig_style)
    fig2=px.histogram(df,x='tenure',color="Churn",title='Predicted churn count by Tenure',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig2.update_layout(fig_style)
    fig3=px.histogram(df,x='Contract',color="Churn",title='Predicted churn by Contract',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig3.update_layout(fig_style)
    fig4=px.histogram(df,x='PaymentMethod',color="Churn",title='Prediction of churn',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig4.update_layout(fig_style)
    #Plots showing expected income loss
    fig5=px.histogram(df,x='MonthlyCharges',y='MonthlyCharges',histfunc='sum',color="Churn",title='Predicted monthly loss by monthly charges',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig5.update_layout(fig_style)
    fig6=px.histogram(df,x='tenure',y='MonthlyCharges',histfunc='sum',color="Churn",title='Predicted monthly loss by Tenure',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig6.update_layout(fig_style)
    fig7=px.histogram(df,x='Contract',y='MonthlyCharges',histfunc='sum',color="Churn",title='Predicted monthly loss by Contract',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig7.update_layout(fig_style)
    fig8=px.histogram(df,x='PaymentMethod',y='MonthlyCharges',histfunc='sum',color="Churn",title='Expected Loss',nbins=30,color_discrete_sequence=['indianred','darkseagreen'],template='plotly_dark')
    fig8.update_layout(fig_style)
    return [kpis, dbc.Row([dbc.Col(dcc.Graph(figure=fig1),width=3,style=graph_style),dbc.Col(dcc.Graph(figure=fig2),width=3,style=graph_style),dbc.Col(dcc.Graph(figure=fig3),width=3,style=graph_style),dbc.Col(dcc.Graph(figure=fig4),width=3,style=graph_style)],class_name='row g-0',),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig5,),width=3,style=graph_style),dbc.Col(dcc.Graph(figure=fig6),width=3,style=graph_style),dbc.Col(dcc.Graph(figure=fig7),width=3,style=graph_style),dbc.Col(dcc.Graph(figure=fig8),width=3,style=graph_style)],class_name='row g-0')]

# this is for future developement
# def create_graph1(data):
#     dff=pd.DataFrame(data)
#     columns_=dff.columns #we will use for drop down for x.
#     element= html.Div([dcc.Dropdown(options=columns_,value=columns_[0]),
#     dcc.Graph(px.histogram(df,x='MonthlyCharges',color="Churn",title='Prediction of churn',nbins=30,color_discrete_sequence=['indianred','darkseagreen']))])
#     return element


### This prints table when store has been updated ###
# @callback(
#     Output('table-placeholder', 'children'),
#     Input('store-prediction', 'data'),
#     prevent_initial_call=True
# )
# def create_graph1(data):
#     dff = pd.DataFrame(data)
#     my_table = dash_table.DataTable(
#         columns=[{"name": i, "id": i} for i in dff.columns],
#         data=dff.to_dict('records'),
#         sort_action='native',
#         filter_action="native"
#     )
#     return my_table

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2022 17:33:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:06] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:06] "GET /_dash-component-suites/dash/dcc/async-upload.js HTTP/1.1" 304 -
127.0.0.1 - - [25/May/2022 17:33:06] "GET /assets/Background.svg HTTP/1.1" 304 -
127.0.0.1 - - [25/May/2022 17:33:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 17:33:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/May/2022 17:33:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -